In [1]:
import pandas as pd
import numpy as np
import requests
# import api keys
from api_keys import *

# Population data

In [2]:
cities = ['Berlin','Paris','Amsterdam','Barcelona','Rome','Lisbon','Prague','Vienna','Madrid']

In [3]:
cities_info = []

for x in cities:
    name = x
    api_url = 'https://api.api-ninjas.com/v1/city?name={}'.format(name)
    response = requests.get(api_url, headers={'X-Api-Key': cities_key})

    cities_info.append(pd.json_normalize(response.json()))

cities_info = pd.concat(cities_info, ignore_index=True)

In [4]:
cities_info = cities_info.assign(municipality_iso_country = lambda x: x['name'] + ',' + x['country'])
cities_info

,name,latitude,longitude,country,population,is_capital,municipality_iso_country
0,Berlin,52.5167,13.3833,DE,3644826,True,"Berlin,DE"
1,Paris,48.8566,2.3522,FR,11020000,True,"Paris,FR"
2,Amsterdam,52.3500,4.9166,NL,1031000,True,"Amsterdam,NL"
3,Barcelona,41.3825,2.1769,ES,4588000,False,"Barcelona,ES"
4,Rome,41.8931,12.4828,IT,2872800,True,"Rome,IT"
5,Lisbon,38.7452,-9.1604,PT,506654,True,"Lisbon,PT"
6,Prague,50.0833,14.4167,CZ,1324277,True,"Prague,CZ"
7,Vienna,48.2083,16.3731,AT,1840573,True,"Vienna,AT"
8,Madrid,40.4189,-3.6919,ES,3266126,True,"Madrid,ES"


In [6]:
cities_info[['latitude', 'longitude', 'population']]  = cities_info[['latitude', 'longitude', 'population']].astype(str)

In [7]:
schema="gans"
host="wbs-project3-db.cwx6am7qyc91.eu-central-1.rds.amazonaws.com"
user="admin"
password="" 
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [8]:
cities_info.to_sql('cities', if_exists='append', con=con, index=False)

# Airports data

In [9]:
airports = (
pd.read_csv('airports.csv')
    .query('type == "large_airport"')
    .filter(['name','latitude_deg','longitude_deg','iso_country','iso_region','municipality','gps_code','iata_code'])
    .rename(columns={'gps_code':'icao_code'})
    .assign(municipality_iso_country = lambda x: x['municipality'] + ',' + x['iso_country'])
)
airports

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
10890,Honiara International Airport,-9.428000,160.054993,SB,SB-CT,Honiara,AGGH,HIR,"Honiara,SB"
12461,Port Moresby Jacksons International Airport,-9.443380,147.220001,PG,PG-NCD,Port Moresby,AYPY,POM,"Port Moresby,PG"
12981,Keflavik International Airport,63.985001,-22.605600,IS,IS-2,Reykjavík,BIKF,KEF,"Reykjavík,IS"
13028,Priština Adem Jashari International Airport,42.572800,21.035801,XK,XK-01,Prishtina,BKPR,PRN,"Prishtina,XK"
17254,Guodu Air Base,36.001741,117.632010,CN,CN-37,"Xintai, Tai'an",NaN,NaN,"Xintai, Tai'an,CN"
...,...,...,...,...,...,...,...,...,...
68223,Chengdu Shuangliu International Airport,30.578501,103.946999,CN,CN-51,Chengdu,ZUUU,CTU,"Chengdu,CN"
68295,Ürümqi Diwopu International Airport,43.907101,87.474197,CN,CN-65,Ürümqi,ZWWW,URC,"Ürümqi,CN"
68306,Harbin Taiping International Airport,45.623402,126.250000,CN,CN-23,Harbin,ZYHB,HRB,"Harbin,CN"
68321,Dalian Zhoushuizi International Airport,38.965698,121.539001,CN,CN-21,"Ganjingzi, Dalian",ZYTL,DLC,"Ganjingzi, Dalian,CN"


In [10]:
airports.dropna().to_sql('airports', if_exists='append', con=con, index=False)